In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [54]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests.packages import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Extract data

Loops through each year from 2016 onwards, extracting data for the 50 winners

In [67]:
# Initialise empty list
df_list = []

for year in range(2016, 2022):
    if year == 2019:
        continue
    print(f'Extracting {year}')
    
    # Extract data from web page
    fast50 = requests.get(f'https://www.deloitte.co.uk/fast50/winners/{year}/', verify=False)
    
    # Parse
    soup = BeautifulSoup(fast50.content, 'html.parser') 
    
    # Find table
    table = soup.find("table", attrs={"class": "block__winners"})

    # Extract body
    table_body = table.find('tbody')

    # Extract data from rows
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

    # Add columns as dataframe
    output_cols = ['Rank', 'Company name', 'Sector', 
                   'Region', 'Growth rate', 'Description']
    df_winners = pd.DataFrame(data=data,
                              columns=output_cols)
    
    # Add year
    df_winners['Year'] = year
    
    # Append
    df_list.append(df_winners)

Extracting 2016
Extracting 2017
Extracting 2018
Extracting 2020
Extracting 2021


ValueError: 6 columns passed, passed data had 300 columns

## Write outputs

In [71]:
pd.concat(df_list).to_csv('fast_50_extract.csv')